# Bonsai Data Analyst Test - Wrangling with Jupyter
#### By Saurabh Sarkar

***

I will be using Jupyter notebooks and relevant Python libraries to perform Exploratory Data Analysis (or EDA) on redacted customer, order and product data. EDA involes exploring and augmenting the data in order to maximize the potential of the analysis to draw conclusions and make predictions. Before exploration, we will WRANGLE the data to ensure its quality. It is in fact common practice to re-visit wrangling throughout the EDA process as new problems are discovered. However, in this report the steps are presented in order to highglight the necessary steps required to go from discovery to exploration to providing validated and actionable insight for the Business to execute.

<ul>
<li><a href="#prelim">Preliminary Analysis</a></li>
<li><a href="#wrangle">Data Wrangling</a></li>
<li><a href="#eda">Exploratory Data Analysis</a></li>
</ul>

With multiple data sources, it is useful to organize the process using Python libraries within Jupyter notebook. It is a complete tool that also includes visulization libraries. However other data analysis tools like *Data Studio*, *R Studio* and *Tableau* are more effective data exploration tools as they have powerful visualization engines providing a more immersive exploration option. The visualization factor plays an important part in justifying and conveying meaning in the insights discovered. Links to the reports from these tools are included in the conclusion section of this report.

Before the analysis begin, I utilized the following question to form a *Mental Model* or a high level view of the requirements of the business decision that needs to be made:
> What are the sales per month

> Highest selling product by category

> Identify customer behaviours

> Are there any operational improvements

<a id='prelim'></a>
## Preliminary Analysis

In this section we load the data and identify cleaning options on the strucutre and itegrity of the datasets. The multiple data sources will also mean opportunities to join the data using any common features that are found in this part of the analysis. 

We start by importing the necessary packages or libraries. The `pandas` and `numpy` packages below are used extensively for data analysis using Python

In [2165]:
# Import Packages
import pandas as pd
import numpy as np
# This allows us to view visualizations in Jupyter notebook
%matplotlib inline


Load csv files into a Pandas dataframe for further analysis. A `index_col=0` parameter is used to disregard the first 'index' column of the underlying csv files

In [2166]:
# Create dataframes from csv files
## Include index_col=0 as underlying csv data includes a index column
## Also explicitly state conversion for 'zipcode' as a string as it is auto converted 
    ## to int and there is risk of losing leading zeroes
custimport = pd.read_csv('./bonsai_mock_data/customer_data.csv', index_col=0, converters={'zipcode': lambda x: str(x)})
orderimport = pd.read_csv('./bonsai_mock_data/order_data.csv', index_col=0)
orderprodimport = pd.read_csv('./bonsai_mock_data/order_product_data.csv', index_col=0)
prodimport = pd.read_csv('./bonsai_mock_data/product_data.csv', index_col=0)

#### Dataset 1 - Customer Data
Consists of customer data indentifying orders to unique customer and their the customers address. This will be a key dataset to explore the behavior of customers.

In [2167]:
# Show first 5 records
custimport.head()

,customer_order_id,customer_unique_id,zipcode
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056


In [2168]:
# Structure of dataset
custimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 3 columns):
customer_order_id     99441 non-null object
customer_unique_id    99441 non-null object
zipcode               99441 non-null object
dtypes: object(3)
memory usage: 3.0+ MB


Discrepencies in the data integrity of zipcode is fortunately visible from the first few records having 4 digits instead of 5. This is probably due to the datatype being set as an integer instead of a string. Good practice is to set zipcode as string as we do not perform any mathematical calculations with it. Also many countries like the UK and Canada is alpha numeric zip codes

#### Dataset 2 - Order Data
The order dataset contains details linked to each order. The fullfillment times/dates can be used to determine potential for operational improvements. Since the orders are also assgined a `customer_order_id` we can reference the `CUSTOMER` dataset and provide deeper customer analysis. 

In [2169]:
# First 5 records
orderimport.head()

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-16 18:17:02


In [2170]:
# Structure of dataset
orderimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 6 columns):
order_id             99441 non-null object
customer_order_id    99441 non-null object
order_status         99441 non-null object
purchased_at         99441 non-null object
approved_at          99281 non-null object
delivered_at         96476 non-null object
dtypes: object(6)
memory usage: 5.3+ MB


The structure of the dataset reveals Null values for `approved_at` and `delivered_at`. Further investigation is required to ensure that these Null values are accurately represented. A valid null value for example could be a NULL delivery date-time when product has NOT been delivered

There are some cases of anomolies evident when looking at the breakdown of `order_status` within a subset of null delivery dates and null approved dates. For example for both subsets it is odd to find orders with a status of 'delivered'. Further investigation and wrangling will be required for this dataset

In [2171]:
# Here, we only look at orders that do not have a 'delivered_at' date
orderimport[orderimport['delivered_at'].isnull()].order_status.value_counts()

shipped        1107
canceled        619
unavailable     609
invoiced        314
processing      301
delivered         8
created           5
approved          2
Name: order_status, dtype: int64

In [2172]:
# And here we look at orders with no 'approved_at' date
orderimport[orderimport['approved_at'].isnull()].order_status.value_counts()

canceled     141
delivered     14
created        5
Name: order_status, dtype: int64

#### Dataset 3 - Order Product Data
The order product dataset links the orders to a specific product and its respective price and shipping cost. This will be the source to capture monetary value for sales and shipping

In [2173]:
# First 5 records
orderprodimport.head()

,order_id,num_items_in_order,product_id,price,shipping
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14


In [2174]:
# Structure of dataset
orderprodimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112650 entries, 0 to 112649
Data columns (total 5 columns):
order_id              112650 non-null object
num_items_in_order    112650 non-null int64
product_id            112650 non-null object
price                 112650 non-null float64
shipping              112650 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 5.2+ MB


No issues with the integrity of the id fields, with all ids have a consistent length of 32 characters. 
> *NOTE: This same method used below was utlized for all other ID fields in the other datasets*

In [2175]:
orderprodimport.order_id.str.len().value_counts()

32    112650
Name: order_id, dtype: int64

In [2176]:
orderprodimport.product_id.str.len().value_counts()

32    112650
Name: product_id, dtype: int64

DUPLICATION analysis

> Note: Duplication assessments using the methods below were also perfomed on the customer and order datasets and have not revealed repeated records. 

The Duplicate records in the `orderproductimport` dataset are evident when the `num_items_in_order` is excluded in the query. This seems to be a result of that field being used to repeat the same iteration of the order whenever the number of items are more than 1.

Using the `duplicated()` function to tag duplicates and using a `keep=False` parameter to ensure the view shows all values, we can see that items in order are captured as a running total for each order. We will perform the clean up changes in the 'Wrangle' section.

In [2177]:
# Out of 112650, there are 17313 'duplicate' records
# .head(10) to only show 10 of those records
orderprodimport[orderprodimport.duplicated(subset=['order_id', 'product_id', 'price' ,'shipping'], keep = False)].info()
orderprodimport[orderprodimport.duplicated(subset=['order_id', 'product_id', 'price' ,'shipping'], keep = False)].head(10)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 17313 entries, 13 to 112643
Data columns (total 5 columns):
order_id              17313 non-null object
num_items_in_order    17313 non-null int64
product_id            17313 non-null object
price                 17313 non-null float64
shipping              17313 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 811.5+ KB


,order_id,num_items_in_order,product_id,price,shipping
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,49.90,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,49.90,13.37
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,21.33,15.10
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,21.33,15.10
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,21.33,15.10
42,001ab0a7578dd66cd4b0a71f5b6e1e41,1,0b0172eb0fd18479d29c3bc122c058c2,24.89,17.63
43,001ab0a7578dd66cd4b0a71f5b6e1e41,2,0b0172eb0fd18479d29c3bc122c058c2,24.89,17.63
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,24.89,17.63
48,001d8f0e34a38c37f7dba2a37d4eba8b,1,e67307ff0f15ade43fcb6e670be7a74c,18.99,7.78
49,001d8f0e34a38c37f7dba2a37d4eba8b,2,e67307ff0f15ade43fcb6e670be7a74c,18.99,7.78


#### Dataset 4 - Product Data
The product dataset consists of more details relating to each product

In [2178]:
# First 5 records
prodimport.head()

,product_id,number_of_photos,category
0,1e9e8ef04dbcff4541ed26657ea517e5,1,perfumery
1,6a2fb4dd53d2cdb88e0432f1284a004c,2,perfumery
2,0d009643171aee696f4733340bc2fdd0,1,perfumery
3,b1eae565a61935e0011ee7682fef9dc9,2,perfumery
4,8da90b37f0fb171b4877c124f965b1f6,3,perfumery


In [2179]:
# Dataset structure
prodimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32328 entries, 0 to 32327
Data columns (total 3 columns):
product_id          32328 non-null object
number_of_photos    32328 non-null int64
category            32328 non-null object
dtypes: int64(1), object(2)
memory usage: 1010.2+ KB


The breakdown for `number_of_photos` and `category` can be seen below, where the same product category has different options with different number of photos

In [2180]:
prodimport.groupby(['category', 'number_of_photos'])['product_id'].count()

category          number_of_photos
air_conditioning  1                    67
                  2                    19
                  3                     7
                  4                    16
                  5                    12
                  6                     2
                  7                     1
art               1                    38
                  2                    14
                  3                     7
                  4                     2
                  5                     5
                  6                     3
                  7                     4
                  10                    1
audio             1                    31
                  2                    10
                  3                     7
                  4                     6
                  5                     2
                  6                     2
auto              1                   747
                  2                   320

In [2181]:
# 41 different 'Category'
print("number of categories: {}".format(prodimport.category.value_counts().count()))
prodimport.category.value_counts()

number of categories: 41


furniture                   5991
home                        3398
sports                      2867
health_beauty               2444
auto                        1900
computers_accessories       1639
toys                        1411
watches                     1329
phones                      1250
miscellaneous                988
baby                         931
perfumery                    868
stationery                   858
fashion_accessories          849
garden_tools                 753
pet_shop                     719
construction_tools           569
electronics                  517
books                        370
music                        365
luggage_accessories          349
games                        317
office_furniture             309
food_drink                   277
fashion_shoes                173
industrials                  142
air_conditioning             124
fashion_male                  95
costruction_tools_garden      88
art                           74
christmas 

Categories for different types of fashion should be combined into a single category

In [2182]:
prodimport.query("category.str.contains('fashion')", engine = 'python')['category'].value_counts()

fashion_accessories        849
fashion_shoes              173
fashion_male                95
fashion_underwear_beach     53
fashion_female              27
fashion_sport               19
fashion_kids                 5
Name: category, dtype: int64

2 of the labels are not categorized properly and are in fact mis-spelled as well

In [2183]:
prodimport.query("category.str.contains('tools')", engine = 'python')['category'].value_counts()

garden_tools                753
construction_tools          569
costruction_tools_garden     88
costruction_tools_tools      39
Name: category, dtype: int64

<a id='wrangle'></a>
## Data Wrangling


Data Wrangling means ensuring the quality of the data. Below are the 4 datasets and a summary of our findings so far and other steps necessary to ensure that we 'Clean' and 'Tidy' up our data sets before further analysis
* `orderprodimport` from order_product_data.csv
    * **Step 1**: Remove repeated records captured for each item iteration within the same order
* `orderimport` from order_data.csv
    * **Step 2**: Change datatype for date-time fields from 'object'/'string' to datetime. This is necessary to perform calculations on dates
    * **Step 3**: Identify and tag anomolies in data related to the different date-time fields -- `purchased_at`, `approved_at` and `delivered_at`
* `custimport` from customer_data.csv
    * No duplicate records, No null values
    * **Step 4**: Change zipcode datatype to 'String' and determine if data is valid
* `prodimport`  from product_data.csv
    * No duplicate records, No null values, Data types assigned correctly
    * **Step 5**: Fix/Combine category names for tools and fashion

#### STEP 1 - Create new dataset from `orderprodimport` by removing duplicates

Before - Duplicated records since items are recorded iteratively. Below you can see an example, where the order_id and product_id repeat records for price and shipping for each itertion of the item

In [2184]:
# And querying a specific 'order_id' results in an iterative log of three records
orderprodimport[orderprodimport.order_id == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,num_items_in_order,product_id,price,shipping
32,00143d0f86d6fbd9f9b38ab440ac16f5,1,e95ee6822b66ac6058e2e4aff656071a,21.33,15.1
33,00143d0f86d6fbd9f9b38ab440ac16f5,2,e95ee6822b66ac6058e2e4aff656071a,21.33,15.1
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,21.33,15.1


Also note that the total number of records before anything is changed is 112650

In [2185]:
orderprodimport.shape

(112650, 5)

After - Create a new data set call `ORDER_PRODUCT` by removing records for earlier iterations of multiple item orders. This is done by using the `drop_duplicates()` function using the parameter `keep = last` to ensure only the highest value for number of items is kept

In [2186]:
# Using subset of all features to identify duplicates 
    # and only keeping the last record of duplicate with highest number of items
order_product = orderprodimport.drop_duplicates(subset=['order_id', 'product_id', 'price' ,'shipping'], keep = 'last')

With the duplicates removed, you see that the number of records is reduced by 225 records and we also confirm that no duplicates exist

In [2187]:
# Records reduced from 112650 to 102425
order_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102425 entries, 0 to 112649
Data columns (total 5 columns):
order_id              102425 non-null object
num_items_in_order    102425 non-null int64
product_id            102425 non-null object
price                 102425 non-null float64
shipping              102425 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.7+ MB


VALIDATION

In [2188]:
# Repeat the duplicated() function to see if duplicates were removed
order_product[order_product.duplicated(subset=['order_id', 'product_id', 'price' ,'shipping'], keep = False)]

,order_id,num_items_in_order,product_id,price,shipping


Also by revisiting the same order id, we see that there is only one record and it captures the highest iteration of `num_items_in_order`

In [2189]:
# And only single record exists with the highest number of items in order per 'order id'
## Repeating same query as done on dataset with duplicates
order_product[order_product.order_id == '00143d0f86d6fbd9f9b38ab440ac16f5']

,order_id,num_items_in_order,product_id,price,shipping
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,21.33,15.1


With the refinement above, we also change the column names of `price` and `shipping` in order to accurately reflect that the value in this field is per item

In [2190]:
# USe rename function
order_product = order_product.rename(columns={"price":"price_per_item", "shipping":"shipping_per_item"})

In [2191]:
# New column names
order_product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 102425 entries, 0 to 112649
Data columns (total 5 columns):
order_id              102425 non-null object
num_items_in_order    102425 non-null int64
product_id            102425 non-null object
price_per_item        102425 non-null float64
shipping_per_item     102425 non-null float64
dtypes: float64(2), int64(1), object(2)
memory usage: 4.7+ MB


#### STEP 2 - Change to datetime datatype of features in `orderimport` dataset
*Note: Option exists to parse dates automatically on import, but has been buggy in the past*

Before: Data type of type 'object' (i.e. 'string') which will not allow to perform date difference calculations

In [2192]:
orderimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 6 columns):
order_id             99441 non-null object
customer_order_id    99441 non-null object
order_status         99441 non-null object
purchased_at         99441 non-null object
approved_at          99281 non-null object
delivered_at         96476 non-null object
dtypes: object(6)
memory usage: 5.3+ MB


After: Change date types

In [2193]:
orderimport.purchased_at = orderimport.purchased_at.astype('datetime64[ns]')
orderimport.approved_at = orderimport.approved_at.astype('datetime64[ns]')
orderimport.delivered_at = orderimport.delivered_at.astype('datetime64[ns]')

VALIDATION

In [2194]:
orderimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 6 columns):
order_id             99441 non-null object
customer_order_id    99441 non-null object
order_status         99441 non-null object
purchased_at         99441 non-null datetime64[ns]
approved_at          99281 non-null datetime64[ns]
delivered_at         96476 non-null datetime64[ns]
dtypes: datetime64[ns](3), object(3)
memory usage: 5.3+ MB


#### STEP 3: Identify anomolies in the order dataset related to logging times for the different date timestamp fields


*ANOMOLY 1*: Order status is NOT `delivered` but has a `delivered_at` value

Orders that have status' as 'canceled' or 'unavilable' or 'invoiced' or 'processing' or 'created' or 'approved' are expected to have NULL value for `delivered_at`. Out of the non-delivered states, the anomoly of having a `delivered_at` date-time, is identified only for the `canceled` state. 

In [2195]:
stutus_with_null_delivered = ['canceled', 'unavailable', 'invoiced', 'processing', 'created', 'approved']
orderimport.query("order_status == @stutus_with_null_delivered & delivered_at.notnull()", engine = 'python')

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
2921,1950d777989f6a877539f53795b4c3c3,1bccb206de9f0f25adc6871a1bcf77b2,canceled,2018-02-19 19:48:52,2018-02-19 20:56:05,2018-03-21 22:03:51
8791,dabf2b0e35b423f94618bf965fcb7514,5cdec0bb8cbdf53ffc8fdc212cd247c6,canceled,2016-10-09 00:56:52,2016-10-09 13:36:58,2016-10-16 14:36:59
58266,770d331c84e5b214bd9dc70a10b829d0,6c57e6119369185e575b36712766b0ef,canceled,2016-10-07 14:52:30,2016-10-07 15:07:10,2016-10-14 15:07:11
59332,8beb59392e21af5eb9547ae1a9938d06,bf609b5741f71697f65ce3852c5d2623,canceled,2016-10-08 20:17:50,2016-10-09 14:34:30,2016-10-19 18:47:43
92636,65d1e226dfaeb8cdc42f665422522d14,70fc57eeae292675927697fe03ad3ff5,canceled,2016-10-03 21:01:41,2016-10-04 10:18:57,2016-11-08 10:58:34
94399,2c45c33d2f9cb8ff8b1c86cc28c11c30,de4caa97afa80c8eeac2ff4c8da5b72e,canceled,2016-10-09 15:39:56,2016-10-10 10:40:49,2016-11-09 14:53:50


The correct expecation of a NULL delevered date-time value is evident from the other 619 records (as seen below)

In [2196]:
# null delivery dates with status as 'canceled'
## needed to specify 'engine' when using logical statements
canceled_and_null = orderimport.query("order_status == 'canceled' & delivered_at.isnull()", engine = 'python')

print("Correctly recorded canceled status with null delivery date: {}".format(canceled_and_null.shape[0]))
print("\nExamples of 5 such records:")
canceled_and_null.head()

Correctly recorded canceled status with null delivery date: 619

Examples of 5 such records:


,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
397,1b9ecfe83cdc259250e1a8aca174f0ad,6d6b50b66d79f80827b6d96751528d30,canceled,2018-08-04 14:29:27,2018-08-07 04:10:26,NaT
613,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,NaT
1058,3a129877493c8189c59c60eb71d97c29,0913cdce793684e52bbfac69d87e91fd,canceled,2018-01-25 13:34:24,2018-01-25 13:50:20,NaT
1130,00b1cb0320190ca0daa2c88b35206009,3532ba38a3fd242259a514ac2b6ae6b6,canceled,2018-08-28 15:26:39,NaT,NaT
1801,ed3efbd3a87bea76c2812c66a0b32219,191984a8ba4cbb2145acb4fe35b69664,canceled,2018-09-20 13:54:16,NaT,NaT


*ANOMOLY 2* : Indications that issues exist with the approval system

First example below, shows Null `approved_at` entries even though item is delivered

In [2197]:
## NOTE: needed to specify 'engine' when using logical statements
orderimport.query("approved_at.isnull() & order_status == 'delivered'", engine = 'python')

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
5323,e04abd8149ef81b95221e88f6ed9ab6a,2127dc6603ac33544953ef05ec155771,delivered,2017-02-18 14:40:00,NaT,2017-03-01 13:25:33
16567,8a9adc69528e1001fc68dd0aaebbb54a,4c1ccc74e00993733742a3c786dc3c1f,delivered,2017-02-18 12:45:31,NaT,2017-03-02 10:05:06
19031,7013bcfc1c97fe719a7b5e05e61c12db,2941af76d38100e0f8740a374f1a5dc3,delivered,2017-02-18 13:29:47,NaT,2017-03-01 08:07:38
22663,5cf925b116421afa85ee25e99b4c34fb,29c35fc91fc13fb5073c8f30505d860d,delivered,2017-02-18 16:48:35,NaT,2017-03-09 07:28:47
23156,12a95a3c06dbaec84bcfb0e2da5d228a,1e101e0daffaddce8159d25a8e53f2b2,delivered,2017-02-17 13:05:55,NaT,2017-03-02 11:09:19
26800,c1d4211b3dae76144deccd6c74144a88,684cb238dc5b5d6366244e0e0776b450,delivered,2017-01-19 12:48:08,NaT,2017-01-30 18:16:01
38290,d69e5d356402adc8cf17e08b5033acfb,68d081753ad4fe22fc4d410a9eb1ca01,delivered,2017-02-19 01:28:47,NaT,2017-03-02 03:41:58
39334,d77031d6a3c8a52f019764e68f211c69,0bf35cac6cc7327065da879e2d90fae8,delivered,2017-02-18 11:04:19,NaT,2017-03-02 16:15:23
48401,7002a78c79c519ac54022d4f8a65e6e8,d5de688c321096d15508faae67a27051,delivered,2017-01-19 22:26:59,NaT,2017-02-06 14:22:19
61743,2eecb0d85f281280f79fa00f9cec1a95,a3d3c38e58b9d2dfb9207cab690b6310,delivered,2017-02-17 17:21:55,NaT,2017-03-03 12:16:03


In the second example, we see records where approved time is after the delivery time

In [2198]:
approve_delay = orderimport.query("approved_at > delivered_at")

print("Number of records that are approved after delivery: {}".format(approve_delay.shape[0]))
print("\nExamples of 5 such records:")
approve_delay.head()

Number of records that are approved after delivery: 61

Examples of 5 such records:


,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
199,58d4c4747ee059eeeb865b349b41f53a,1755fad7863475346bc6c3773fe055d3,delivered,2018-07-21 12:49:32,2018-07-26 23:31:53,2018-07-25 23:58:19
483,4df92d82d79c3b52c7138679fa9b07fc,ba0660bf3fffe505ee892e153a2fbd49,delivered,2018-07-24 11:32:11,2018-07-29 23:30:52,2018-07-27 18:55:57
1986,6e57e23ecac1ae881286657694444267,2dda54e25d0984e12705c84d4030e6e0,delivered,2018-08-09 17:36:47,2018-08-20 15:55:42,2018-08-17 16:45:45
3659,f222c56f035b47dfa1e069a88235d730,b74ca180d63f9ae0443e4e13a2f5bdaf,delivered,2018-01-30 09:43:45,2018-02-04 23:31:47,2018-02-01 20:10:38
11738,cf72398d0690f841271b695bbfda82d2,2b7fff075bda701552485ef3f0810257,delivered,2017-09-01 18:45:33,2017-09-13 22:04:39,2017-09-11 14:15:02


And the final example, where approval occurs instantaneously after purchase

In [2199]:
immediate_approval = orderimport.query("approved_at == purchased_at")

print("Number of records that are approved after delivery: {}".format(immediate_approval.shape[0]))
print("\nExamples of 5 such records:")
immediate_approval.head()

Number of records that are approved after delivery: 1296

Examples of 5 such records:


,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at
58,a685d016c8a26f71a0bb67821070e398,911e4c37f5cafe1604fe6767034bf1ae,delivered,2017-03-13 18:14:36,2017-03-13 18:14:36,2017-04-06 13:37:16
182,75351e48296ef42211a0b80c427aae57,cf9e2b07f78cce347089900f49fb4746,delivered,2017-03-14 16:25:43,2017-03-14 16:25:43,2017-04-03 13:37:42
200,a840a7a89e3d9137358eb7b9408681fe,5e320e6a4f03e2e74bc22e1158237fbe,delivered,2017-03-09 22:17:49,2017-03-09 22:17:49,2017-03-16 08:24:47
380,7e05645577366863a93350aac5cc7de5,21809d6b8acb4ca628358610aea9435c,delivered,2017-03-22 17:25:28,2017-03-22 17:25:28,2017-04-06 10:35:15
489,d83706c29baf36eedf5e8adfb0da304e,26545a1a15e77138e98380c49a76053a,delivered,2017-03-13 11:36:11,2017-03-13 11:36:11,2017-03-27 14:12:27


NO CORRUPT DATA TO EXCLUDE: Queries below confirm that the time logging is limited to the approval system and MAJOR discrepencies DO NOT exist

In [2200]:
# No delivery dates before purchase date
orderimport.query("delivered_at < purchased_at")

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at


In [2201]:
# No approval dates before purchase dates
orderimport.query("approved_at < purchased_at")

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at


*TAG ALL ANOMOLIES* - Next step is to take all the findings above and create a copy called `ORDER_TAGGED` dataset with a new 'tag' field identifying the different anomolies
> NOTE: This is good practice to tag items to be discussed with business as well as provide any necessary data that ML engine could use

In [2202]:
# Start by making a copy
order = orderimport.copy()

In [2203]:
# Create an empty field to then add tags to
order['tag'] = np.nan

In [2204]:
order.head()

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at,tag
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,NaN
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,NaN
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29,NaN
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42,NaN
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-16 18:17:02,NaN


In [2205]:
# This will tag approvals that happen at the same time as purchase
order.loc[order.approved_at == order.purchased_at, 'tag'] = 'approval_issue approval-same-as-purchase'

In [2206]:
# This will tag approvals that happened after delivery
order.loc[order.approved_at > order.delivered_at, 'tag'] = 'approval_issue approval-after-delivery'

In [2207]:
# not using dot notation so it is easier to understand
# also putting conditions in parenthesis so that they get evaluated first
## SAME AS: order.query("order_status == 'canceled' & delivered_at.notnull()", engine = 'python')

order.loc[(order['delivered_at'].notnull()) 
                 & (order['order_status'] == 'canceled'), 'tag'] = 'delivery_issue delivered-but-cancelled'

In [2208]:
# Same as evaluating order.query("approved_at.isnull() & order_status == 'delivered'", engine = 'python')
## And because using null functions behave differently, need to put them in parenthesis to ensure that 
    ## it evaluates first and query is correct
    
order.loc[(order['approved_at'].isnull()) 
                 & (order['order_status'] == 'delivered'), 'tag'] = 'approval_issue no-approval-before-delivery'

Final break down of the tagged items

In [2209]:
order.tag.value_counts()

approval_issue approval-same-as-purchase      1296
approval_issue approval-after-delivery          61
approval_issue no-approval-before-delivery      14
delivery_issue delivered-but-cancelled           6
Name: tag, dtype: int64

**STEP 4**: See if zipcode field in `custimport` dataset can be fixed with leading zero - otherwise remove

In [2210]:
custimport.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 99441 entries, 0 to 99440
Data columns (total 3 columns):
customer_order_id     99441 non-null object
customer_unique_id    99441 non-null object
zipcode               99441 non-null object
dtypes: object(3)
memory usage: 3.0+ MB


You can see below that we have quite a significant number of zipcodes with 4 characters. Googling zip codes with and without leading zeros does not provide a clear pattern to fix this field. For this reason we cannot use zip code to accurately represent the address of the customers

In [2211]:
custimport.zipcode.str.len().value_counts()

5    75446
4    23995
Name: zipcode, dtype: int64

Diving a little deeper into the top occurances amongst 4-digit zip codes further confirms the inability of this field to represent customer address
* 07600 is in France, 7600 is in South Africa
* 09371 is in Sao Paolo, 9731 is in Hungary
* 06900 is in Turkey, 6900 is in Switzerland

In [2212]:
#Most common 4 igit zip codes
custimport[custimport.zipcode.str.len() == 4].zipcode.value_counts().head(3)

7600    75
9371    56
6900    46
Name: zipcode, dtype: int64

Without more address information it is not possible to determine if data was ommitted as a leading zero issue or if these addresses are in fact international. 

For now we will not make any changes and determine in EDA step if a correlation to shipping rates can be made

In [2213]:
customer = custimport.copy()

In [2214]:
# Verify drop
customer.head()

,customer_order_id,customer_unique_id,zipcode
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056


**Step 5**: Consolidate 'fashion' category and organize fix assignment of misspelled categories in `prodimport` dataset

In [2215]:
#start by making a copy
product = prodimport.copy()

In [2216]:
product.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32328 entries, 0 to 32327
Data columns (total 3 columns):
product_id          32328 non-null object
number_of_photos    32328 non-null int64
category            32328 non-null object
dtypes: int64(1), object(2)
memory usage: 1010.2+ KB


CHANGE 1 - Before: Fashion is categorized into 7 different sub categories

In [2217]:
product[product.category.str.contains('fashion')]['category'].value_counts()

fashion_accessories        849
fashion_shoes              173
fashion_male                95
fashion_underwear_beach     53
fashion_female              27
fashion_sport               19
fashion_kids                 5
Name: category, dtype: int64

After: Label all instead just as 'fashion'

In [2218]:
product.loc[product.category.str.contains('fashion'), 'category'] = 'fashion'
product[product.category.str.contains('fashion')]['category'].value_counts()

fashion    1221
Name: category, dtype: int64

CHANGE 2 - Before: 2 misspelled categories

In [2219]:
product[product.category.str.contains('tools')]['category'].value_counts()

garden_tools                753
construction_tools          569
costruction_tools_garden     88
costruction_tools_tools      39
Name: category, dtype: int64

AFTER: consolidate accordingly into `garden_tools` and `construction_tools`

In [2220]:
product.loc[product.category == 'costruction_tools_garden', 'category'] = 'garden_tools'
product.loc[product.category == 'costruction_tools_tools', 'category'] = 'construction_tools'

Verify

In [2221]:
product[product.category.str.contains('tools')]['category'].value_counts()

garden_tools          841
construction_tools    608
Name: category, dtype: int64

<a id='eda'></a>
## Exploratory Data Analysis

In the previous Wrangling step the quality and structure of the data was assessed and improved. Now in the EDA step we can now be more elaborate with our assessments. This means to find patterns in your data and to even visualize those relationships and build INTUITION about what you are working with.

<a id='augment'></a>
### Augment and Explore

It is important to AUGMENT the data before we start visualizating them. Below, we can see a summary of the wrangling steps so far and the steps we can take to Augment the data:
* `ORDER_PRODUCT` created by removing duplicates from `ORDERIMPORT`(order_product_data.csv)
    * **Calculations**: Create new fields using calculations using price, shipping, and number of items
    * **Possible Joins**: Merge data with `ORDER_TAGGED` using `order_id`  
    * **Possible Joins**: Merge data with `PRODUCT` using `product_id`
* `ORDER` created by adding tags to identify possible operational anomolies from `ORDERIMPORT`(order_data.csv)
    * **Calculations**: Determine fulfilment times by performing time difference calculations between approval times and delivery times
    * **Possible Joins**: Merge data with `CUSTOMER_NOZIP` using `customer_order_id`
* `CUSTOMER` created by removing zipcode field from `CUSTIMPORT` (customer_data.csv)
* `PRODUCT`  from product_data.csv --  no changes
* **EXPORT**: Augment data further and *EXPORT* into different datasets for exploration

**Calculation Step** - Engineer features based on calculations within `ORDER_PRODUCT` dataset

In [2222]:
order_product.head()

,order_id,num_items_in_order,product_id,price_per_item,shipping_per_item
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14


Create a field to determine the total price calculated based on the sum of price and shipping as a multiple of the number of items in the order

In [2223]:
order_product['total_item_price'] = order_product['num_items_in_order']*order_product['price_per_item']
order_product['total_shipping_price'] = order_product['num_items_in_order']*order_product['shipping_per_item']
order_product['total_order_value'] = order_product['total_item_price'] + order_product['total_shipping_price']

`ORDER_PRODUCT` dataset with the new calculated fields

In [2224]:
# Showing records for when items are more than 1 to confirm calculation
print("Showing new calculations for a sample of records with item count more than 1")
order_product[order_product.num_items_in_order >= 2].head()

Showing new calculations for a sample of records with item count more than 1


,order_id,num_items_in_order,product_id,price_per_item,shipping_per_item,total_item_price,total_shipping_price,total_order_value
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,49.90,13.37,99.80,26.74,126.54
34,00143d0f86d6fbd9f9b38ab440ac16f5,3,e95ee6822b66ac6058e2e4aff656071a,21.33,15.10,63.99,45.30,109.29
44,001ab0a7578dd66cd4b0a71f5b6e1e41,3,0b0172eb0fd18479d29c3bc122c058c2,24.89,17.63,74.67,52.89,127.56
49,001d8f0e34a38c37f7dba2a37d4eba8b,2,e67307ff0f15ade43fcb6e670be7a74c,18.99,7.78,37.98,15.56,53.54
76,002c9def9c9b951b1bec6d50753c9891,2,2d9ff06c8870a518f5f6909774e140fb,78.00,8.90,156.00,17.80,173.80


Also validate for 1 item count and drop irrelavant columns

In [2225]:
order_product[order_product.num_items_in_order == 1].head()

,order_id,num_items_in_order,product_id,price_per_item,shipping_per_item,total_item_price,total_shipping_price,total_order_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14,199.90,18.14,218.04


In [2226]:
order_product.drop(['price_per_item', 'shipping_per_item'], axis = 1, inplace = True)

In [2227]:
order_product.head()

,order_id,num_items_in_order,product_id,total_item_price,total_shipping_price,total_order_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,58.90,13.29,72.19
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,239.90,19.93,259.83
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,199.00,17.87,216.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,12.99,12.79,25.78
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,199.90,18.14,218.04


**Calculation Step** - Calculate time between orders and deliveries in `ORDER` dataset

In [2228]:
# Only keep date portion and remove datetime portion
order['purchase_date'] = pd.to_datetime(order.purchased_at).dt.date

In [2229]:
order.head()

,order_id,customer_order_id,order_status,purchased_at,approved_at,delivered_at,tag,purchase_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-10 21:25:13,NaN,2017-10-02
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-08-07 15:27:45,NaN,2018-07-24
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-17 18:06:29,NaN,2018-08-08
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-12-02 00:28:42,NaN,2017-11-18
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-16 18:17:02,NaN,2018-02-13


In [2230]:
# Include elasped time between order and delivery
## astype ensures that difference is calculated in seconds
order['approval_in-hours'] = (order.approved_at - order.purchased_at).astype('timedelta64[s]')/3600
order['delivery_in-hours'] = (order.delivered_at - order.purchased_at).astype('timedelta64[s]')/3600

Also drop original date fields as we have calculated the necessary features

In [2231]:
drop_columns = ['purchased_at', 'approved_at', 'delivered_at']
order.drop(drop_columns, axis = 1, inplace = True)

`ORDER` dataset with new calculated fields

In [2232]:
order.head()

,order_id,customer_order_id,order_status,tag,purchase_date,approval_in-hours,delivery_in-hours
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,NaN,2017-10-02,0.178333,202.477778
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,NaN,2018-07-24,30.713889,330.768889
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,NaN,2018-08-08,0.276111,225.461111
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,NaN,2017-11-18,0.298056,317.010000
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,NaN,2018-02-13,1.030556,68.973056


**Merge Step 1** - Provide more context to product IDs in `ORDER_PRODUCTS` by merging product data (`category` and `number_of_photos`) from `PRODUCTS` on `product_id`
* Output 1 - `MASTER_NONULL` as a inner join/merge resulting in 1482 fewer records but no missing metrics
* Output 2 - `MASTER` using left join to capture product Ids that do not have a category, resulting in a complete join grabbing all records, but will consist of `Null` metrics

Merging on a 'Inner Join' results in a loss of records from 102425 to 100943

In [2233]:
# Original number of rows to add product data to
order_product.shape

(102425, 6)

In [2234]:
# Merged dataset will be an enhancement of the original order_product
master_nonull = order_product.merge(product, on = 'product_id')

In [2235]:
# master_nonull has fewer records
master_nonull.shape

(100943, 8)

Fewer records in the merged data set of `master_nonull` implies that product IDs in `ORDER_PRODUCT` dataset do not exist in the `PRODUCT` dataset. 

To account for these missing categories, another dataset called `MASTER` will be created using a 'LEFT JOIN' resulting in the original number of 102425 records but with some NULL product features due to missing details for certain product IDs

In [2236]:
#Start by renaming the common 'product_id' field in both datasets
product.rename(columns={'product_id':'product_id_r'}, inplace = True)
order_product.rename(columns={'product_id':'product_id_l'}, inplace = True)

In [2237]:
#Then do a left join maintaining records on order_product_dedup even if they do not match
master = order_product.merge(product, left_on = 'product_id_l', right_on = 'product_id_r', how = 'left')

VALIDATION: Original number of records of 102425

In [2238]:
master.shape

(102425, 9)

Out of the 1482 records, we can identify 623 unique products with no product categorization!

In [2239]:
query_nullproductcategory = master[master.product_id_r.isnull()]
print("Count of unmatched records: {}".format(query_nullproductcategory.shape[0]))
print("\nExample of first 5 records")
query_nullproductcategory.head()

Count of unmatched records: 1482

Example of first 5 records


,order_id,num_items_in_order,product_id_l,total_item_price,total_shipping_price,total_order_value,product_id_r,number_of_photos,category
113,0046e1d57f4c07c8c92ab26be8c3dfc0,1,ff6caf9340512b8bf6d2a2a6df032cfa,7.79,7.78,15.57,NaN,NaN,NaN
115,00482f2670787292280e0a8153d82467,1,a9c404971d1a5b1cbc2e4070e02731fd,7.60,10.96,18.56,NaN,NaN,NaN
122,004f5d8f238e8908e6864b874eda3391,1,5a848e4ab52fd5445cdc07aab1c40e48,122.99,15.61,138.60,NaN,NaN,NaN
128,0057199db02d1a5ef41bacbf41f8f63b,1,41eee23c25f7a574dfaf8d5c151dbb12,20.30,16.79,37.09,NaN,NaN,NaN
154,006cb7cafc99b29548d4f412c7f9f493,1,e10758160da97891c2fdcbc35f0f031d,56.00,14.14,70.14,NaN,NaN,NaN


In [2240]:
query_missing_product_unique = master[master.product_id_r.isnull()].product_id_l.value_counts()
print("Top 10 products amongst the {} products that have missing details".format(len(query_missing_product_unique)))
query_missing_product_unique.head(10)

Top 10 products amongst the 623 products that have missing details


5a848e4ab52fd5445cdc07aab1c40e48    194
b1d207586fca400a2370d50a9ba1da98     43
76d1a1a9d21ab677a61c3ae34b1b352f     32
ad88641611c35ebd59ecda07a9f17099     28
3b60d513e90300a4e9833e5cda1f1d61     28
4914f8796af2ecd359fd8f44b9b92339     23
0502d1a36be75bd36b452f31c6ed264a     21
e0f33a3329af6716a0bb47fd7a664439     20
b36f3c918c91478c4559160022d3f14e     17
4e0d588f8e002f2bad9cbe0b8f66f6f6     15
Name: product_id_l, dtype: int64

**Merge Step 2** - Include further order details to `MASTER` and `master_nonull` by merging with `ORDER` dataset

We start by verifying that all order records are unique in the `ORDER` dataset

In [2241]:
combined_dup = order.duplicated(subset = ['order_id', 'customer_order_id'], keep = False)
order_dup = order.duplicated('order_id', keep = False)
customer_dup = order.duplicated('customer_order_id', keep = False)
print("duplicated records: {}, repeated order_ids: {}, repeated customer_order_ids: {}"
      .format(order[combined_dup].shape[0],order[order_dup].shape[0],order[customer_dup].shape[0]))

duplicated records: 0, repeated order_ids: 0, repeated customer_order_ids: 0


After the merge both `MASTER` and `master_nonull` have order details as well

In [2242]:
master = master.merge(order, on = 'order_id')
master_nonull = master_nonull.merge(order, on = 'order_id')

VALIDATION

In [2243]:
# Verify that inner merge resulted in no missing records
print("For order dataset with null product category, we expect 102425 records and we got {} records". format(master.shape[0]))
print("For order dataset with no nulls we expect 100943 records and we got {} records".format(master_nonull.shape[0]))

For order dataset with null product category, we expect 102425 records and we got 102425 records
For order dataset with no nulls we expect 100943 records and we got 100943 records


**Merge Step 3** - Include further details in the form of customer features by merging with `CUSTOMER` dataset
* OUTPUT 1 - Dataset `MASTER` with customer details
* OUTPUT 2 - Dataset `MASTER_nonull` with customer details

The uniqueness of the ID fields in `CUSTOMER` dataset reveal that `customer_order_id` that we are going to join/merge with, is unique

In [2244]:
combined_dup = customer.duplicated(subset = ['customer_order_id', 'customer_unique_id'], keep = False)
order_dup = customer.duplicated('customer_order_id', keep = False)
customer_dup = customer.duplicated('customer_unique_id', keep = False)
print("duplicated records: {}, repeated customer_order_ids: {}, repeated customer_ids: {}"
      .format(customer[combined_dup].shape[0],customer[order_dup].shape[0],customer[customer_dup].shape[0]))

duplicated records: 0, repeated customer_order_ids: 0, repeated customer_ids: 6342


Looking at the customer dataset further we see that out of the 99441 unique records, there are 6342 unique customers

In [2245]:
# Number of unique customers
## Note: unique customer count is taken using the variables created in above code
print("The total of {} order records, were ordered by {} unique customers".\
      format(customer.shape[0], customer[customer_dup].shape[0]))

print("\nOut of which these top 10 most common occurances of customers")
customer.customer_unique_id.value_counts().head(10)

The total of 99441 order records, were ordered by 6342 unique customers

Out of which these top 10 most common occurances of customers


8d50f5eadf50201ccdcedfb9e2ac8455    17
3e43e6105506432c953e165fb2acf44c     9
6469f99c1f9dfae7733b25662e7f1782     7
1b6c7548a2a1f9037c1fd3ddfed95f33     7
ca77025e7201e3b30c44b472ff346268     7
47c1a3033b8b77b3ab6e109eb4d5fdf3     6
63cfc61cee11cbe306bff5857d00bfe4     6
12f5d6e1cbf93dafd9dcc19095df0b3d     6
de34b16117594161a6a89c50b289d35a     6
dc813062e0fc23409cd255f7f53c7074     6
Name: customer_unique_id, dtype: int64

MERGE AND VALIDATION

In order to get a deeper understanding of these customers we will need to merge this data with the previous merges that combined order and product data

In [2246]:
# We start with a simpler inner join, where the resulting dataset has records with no null values
print("MERGE - Augment 'master_nonull' dataset with customer information")
print("\nBefore merge `master_nonull has {} records and {} fields".format(master_nonull.shape[0], master_nonull.shape[1]))
# Perform merge
master_nonull = master_nonull.merge(customer, on = 'customer_order_id')
print("\nAfter merge 'master_nonull' dataset augmented with customer info {} records and {} fields".format(master_nonull.shape[0], master_nonull.shape[1]))

MERGE - Augment 'master_nonull' dataset with customer information

Before merge `master_nonull has 100943 records and 14 fields

After merge 'master_nonull' dataset augmented with customer info 100943 records and 16 fields


In [2247]:
# We also create a master dataset that consists of all records including the ones that are missing product features
print("MERGE - Augment 'master' dataset with customer information")
print("\nBefore merge 'master' has {} records and {} fields".format(master.shape[0], master.shape[1]))
# Perform merge
master = master.merge(customer, on = 'customer_order_id')
print("\nAfter merge 'master' dataset augmented with customer info {} records and {} fields".format(master.shape[0], master.shape[1]))

MERGE - Augment 'master' dataset with customer information

Before merge 'master' has 102425 records and 15 fields

After merge 'master' dataset augmented with customer info 102425 records and 17 fields


**Subset Augmentation**: Look at the final combined dataset and identify opportunities to create new datasets
* `CUSTOMER_RETENTION`

The `MASTER` dataset so far consists of repeated occurances of products purchased in the same `customer_order_id`

In [2248]:
master.groupby(['customer_unique_id', 'customer_order_id', 'purchase_date'])['purchase_date']\
.count().sort_values(ascending = False).head(10)

customer_unique_id                customer_order_id                 purchase_date
c8ed31310fc440a3f8031b177f9842c3  0d861a5e4dd6a9079d89e1330848f0ab  2018-08-12       8
bf869f6a89c8ba217f47e22359f884f2  1205480caca6c37f55954da838933b8a  2018-08-16       7
595e38fad1949e25468ad1c7c06924d0  a67a246af6ba598a14cc86df3c0354ee  2018-05-12       7
33de26d1fafbfd4945eb586f7136efe6  30bb84b541c96af98ba7d90b9ebf35d0  2017-11-21       7
be160ff1d833563ea876b3662de0a653  6c44a903274653cddb1df3bcb05ac71f  2018-07-08       6
45a529e4af8f502a189094332e8ed24e  8c4bcd3c3737198f62421c18e0348554  2017-01-15       6
8dc697d03f771cecc2534534a73eaaf9  e898b5ef24833b9cb9e2d4f00b937595  2018-08-24       6
fe86d9409d83a3c561ce16e64d2d55e6  223f5e3208f2fa202a8ff706a4d9b206  2017-10-17       6
83c6462cf9b51c26d6102ac0c48a1100  c9b6ccc9f42e3ac60bb09a0f53a09973  2018-03-25       6
d97b3cfb22b0d6b25ac9ed4e9c2d481b  be1c4e52bb71e0c54b11a26b8e8d59f2  2017-10-17       6
Name: purchase_date, dtype: int64

In order to effectivley analyze retention, customer records per order id should be represented on a single line. As we can see for the individual records for a multiply occuring customer id, the records for a particular `customer_order_id`, share the same `order_status`, `purchase_date`, and delivery and approval times

In [2249]:
master[master.customer_unique_id == 'bf869f6a89c8ba217f47e22359f884f2']

,order_id,num_items_in_order,product_id_l,total_item_price,total_shipping_price,total_order_value,product_id_r,number_of_photos,category,customer_order_id,order_status,tag,purchase_date,approval_in-hours,delivery_in-hours,customer_unique_id,zipcode
47837,77df84f9195be22a4e9cb72ca9e8b4c2,1,49650e49c3af7ccc896f634d039cd921,45.00,3.28,48.28,49650e49c3af7ccc896f634d039cd921,5.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47838,77df84f9195be22a4e9cb72ca9e8b4c2,2,daef10c972d7295283e377d5fd9985f1,37.80,6.56,44.36,daef10c972d7295283e377d5fd9985f1,1.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47839,77df84f9195be22a4e9cb72ca9e8b4c2,3,9474f3c0da0c4fd0bdf88cf6ac1b5d04,117.00,9.84,126.84,9474f3c0da0c4fd0bdf88cf6ac1b5d04,3.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47840,77df84f9195be22a4e9cb72ca9e8b4c2,4,0e35a413d832e63c5d46101b7c88de0f,99.96,13.12,113.08,0e35a413d832e63c5d46101b7c88de0f,1.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47841,77df84f9195be22a4e9cb72ca9e8b4c2,5,5d1d8ec5b31c5c359f10c4b23b819170,104.50,16.50,121.00,5d1d8ec5b31c5c359f10c4b23b819170,1.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47842,77df84f9195be22a4e9cb72ca9e8b4c2,6,154696c4f8e96ed48f0c191eb09c0683,113.40,19.68,133.08,154696c4f8e96ed48f0c191eb09c0683,1.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075
47843,77df84f9195be22a4e9cb72ca9e8b4c2,7,8ccaa3f8b171b199e37294ece30815a3,132.30,22.96,155.26,8ccaa3f8b171b199e37294ece30815a3,1.0,furniture,1205480caca6c37f55954da838933b8a,delivered,NaN,2018-08-16,0.235278,32.856944,bf869f6a89c8ba217f47e22359f884f2,11075


This means that on grouping the only way to keep product features is  by `concatenating` DISTINCT `categories` and `number_of_photos` into a single field. We are maintaining another dataset `MASTER_nonull` that has the product features and no null values, so we can safely remove those features when combining customers into a single line record

Lets create a copy called `CUSTOMER_RETENTION` from master

In [2250]:
# master_raw will track all data
customer_retention = master.copy()

Also need to change `tag` values from null so that it can be included in the dataset created using `groupby`

In [2251]:
customer_retention.loc[customer_retention.tag.isnull(), 'tag'] = 'no anomolies'

In [2252]:
# for the new datast, keep fields that are consistent for customer_order_id
common_columns = ['customer_unique_id', 'customer_order_id', 'purchase_date', 'order_status', \
                'approval_in-hours', 'delivery_in-hours', 'tag']

In [2253]:
# Group by common fields and do a count and sum of total order value
customer_retention = customer_retention.groupby(common_columns)\
                    .agg({'zipcode':'count', 'num_items_in_order':'sum', 'total_item_price':'sum', \
                          'total_shipping_price':'sum', 'total_order_value':'sum'})\
                    .reset_index()\
                    .rename(columns={'zipcode':'number_item_types'})

Note that we have created a new feature called `number_item_types` that represent the different records in the master dataset. Here we are still keeping track of it as a single digit

In [2254]:
customer_retention[customer_retention.customer_unique_id == 'bf869f6a89c8ba217f47e22359f884f2']

,customer_unique_id,customer_order_id,purchase_date,order_status,approval_in-hours,delivery_in-hours,tag,number_item_types,num_items_in_order,total_item_price,total_shipping_price,total_order_value
72309,bf869f6a89c8ba217f47e22359f884f2,1205480caca6c37f55954da838933b8a,2018-08-16,delivered,0.235278,32.856944,no anomolies,7,28,649.96,91.94,741.9


The resulting `customer retetion` dataset is still not good enough to accurately identify returning customers, as instances exist where different order IDs for the same customer occur on the same day (see below)

In [2255]:
customer_retention[customer_retention.customer_unique_id == '12f5d6e1cbf93dafd9dcc19095df0b3d']

,customer_unique_id,customer_order_id,purchase_date,order_status,approval_in-hours,delivery_in-hours,tag,number_item_types,num_items_in_order,total_item_price,total_shipping_price,total_order_value
7188,12f5d6e1cbf93dafd9dcc19095df0b3d,27129ce1ebca4ffa23a049c51cc8ec95,2017-01-05,delivered,36.175556,288.743333,no anomolies,1,1,6.9,8.72,15.62
7189,12f5d6e1cbf93dafd9dcc19095df0b3d,588047d7101d88c333691e47659d7099,2017-01-05,delivered,37.339444,288.840556,no anomolies,1,1,10.9,8.72,19.62
7190,12f5d6e1cbf93dafd9dcc19095df0b3d,6152d0774bbbf74f7140541c0569dafa,2017-01-05,delivered,37.368611,266.180556,no anomolies,1,1,10.9,8.72,19.62
7191,12f5d6e1cbf93dafd9dcc19095df0b3d,a969b9f8d923bc7fd97b578f7c499194,2017-01-05,delivered,37.436944,290.495000,no anomolies,1,1,9.9,8.72,18.62
7192,12f5d6e1cbf93dafd9dcc19095df0b3d,c0352e94059e3e5a714c9ad0c8306a54,2017-01-05,delivered,36.744722,264.552500,no anomolies,1,1,9.9,8.72,18.62
7193,12f5d6e1cbf93dafd9dcc19095df0b3d,ed1793d2d1e4175d5846ce7ebb4a01f5,2017-01-05,delivered,36.321667,263.882778,no anomolies,1,1,9.9,8.72,18.62


One way to resolve this would be to group only by customer and purchase date and then TAG retention count to the customer ID and join that field to `CUSTOMER RETENTION`

We resolve this by further grouping by the customer id and purchase date, while tracking the different customer order Id as a count

In [2256]:
# start with another copy
outer_group = customer_retention.copy()

In [2257]:
# Then group by customer id and date
outer_group = outer_group.groupby(['customer_unique_id','purchase_date'])\
                    .agg({'order_status':'count', 'total_order_value':'sum'})\
                    .reset_index()\
                    .rename(columns={'order_status':'number_order_types'})

Repeat the same query to verify that customer orders on same date are now combined and tracked using `number_order_types`

In [2258]:
outer_group[outer_group.customer_unique_id == '12f5d6e1cbf93dafd9dcc19095df0b3d']

,customer_unique_id,purchase_date,number_order_types,total_order_value
7122,12f5d6e1cbf93dafd9dcc19095df0b3d,2017-01-05,6,110.72


We can now differentiate returning customers as customers who make purchases on different dates. 

In [2259]:
outer_group.groupby('customer_unique_id')['purchase_date'].count().sort_values(ascending = False).head(10)

customer_unique_id
8d50f5eadf50201ccdcedfb9e2ac8455    15
ca77025e7201e3b30c44b472ff346268     7
1b6c7548a2a1f9037c1fd3ddfed95f33     7
3e43e6105506432c953e165fb2acf44c     6
dc813062e0fc23409cd255f7f53c7074     6
63cfc61cee11cbe306bff5857d00bfe4     6
f0e310a6839dce9de1638e0fe5ab282a     6
6469f99c1f9dfae7733b25662e7f1782     6
47c1a3033b8b77b3ab6e109eb4d5fdf3     5
fe81bb32c243a86b2f86fbf053fe6140     5
Name: purchase_date, dtype: int64

We need to group again to create a TAG to identify number of retentions

In [2260]:
inner_group = outer_group.groupby('customer_unique_id')['purchase_date'].count().reset_index()\
                                                    .rename(columns={'purchase_date':'retention_count'})

`retention_count` higher than 1 identifies returning customers

In [2261]:
inner_group.sort_values('retention_count', ascending = False).head(10)

,customer_unique_id,retention_count
51424,8d50f5eadf50201ccdcedfb9e2ac8455,15
73910,ca77025e7201e3b30c44b472ff346268,7
10058,1b6c7548a2a1f9037c1fd3ddfed95f33,7
87870,f0e310a6839dce9de1638e0fe5ab282a,6
80525,dc813062e0fc23409cd255f7f53c7074,6
36496,63cfc61cee11cbe306bff5857d00bfe4,6
36702,6469f99c1f9dfae7733b25662e7f1782,6
22775,3e43e6105506432c953e165fb2acf44c,6
81167,de34b16117594161a6a89c50b289d35a,5
34585,5e8f38a9a1c023f3db718edcf926a2db,5


Also verify that there are no duplicated ids

In [2262]:
inner_group[inner_group.customer_unique_id.duplicated() == True]

,customer_unique_id,retention_count


With a retention count attached to each customer we can merge this data to the `CUSTOMEER_RETENTION` data

In [2263]:
customer_retention.shape

(96462, 12)

Perform `merge`

In [2264]:
customer_retention = customer_retention.merge(inner_group, on='customer_unique_id')

VALIDATION

In [2265]:
# Verify we have same number of records
customer_retention.shape

(96462, 13)

And when we check customer_retention now, we see in the immediate example of different orders on the same day is not marked as an retention

While orders placed in different dates are marked as retention

In [2266]:
# Not marked as retention
customer_retention[customer_retention.customer_unique_id == '12f5d6e1cbf93dafd9dcc19095df0b3d']

,customer_unique_id,customer_order_id,purchase_date,order_status,approval_in-hours,delivery_in-hours,tag,number_item_types,num_items_in_order,total_item_price,total_shipping_price,total_order_value,retention_count
7188,12f5d6e1cbf93dafd9dcc19095df0b3d,27129ce1ebca4ffa23a049c51cc8ec95,2017-01-05,delivered,36.175556,288.743333,no anomolies,1,1,6.9,8.72,15.62,1
7189,12f5d6e1cbf93dafd9dcc19095df0b3d,588047d7101d88c333691e47659d7099,2017-01-05,delivered,37.339444,288.840556,no anomolies,1,1,10.9,8.72,19.62,1
7190,12f5d6e1cbf93dafd9dcc19095df0b3d,6152d0774bbbf74f7140541c0569dafa,2017-01-05,delivered,37.368611,266.180556,no anomolies,1,1,10.9,8.72,19.62,1
7191,12f5d6e1cbf93dafd9dcc19095df0b3d,a969b9f8d923bc7fd97b578f7c499194,2017-01-05,delivered,37.436944,290.495000,no anomolies,1,1,9.9,8.72,18.62,1
7192,12f5d6e1cbf93dafd9dcc19095df0b3d,c0352e94059e3e5a714c9ad0c8306a54,2017-01-05,delivered,36.744722,264.552500,no anomolies,1,1,9.9,8.72,18.62,1
7193,12f5d6e1cbf93dafd9dcc19095df0b3d,ed1793d2d1e4175d5846ce7ebb4a01f5,2017-01-05,delivered,36.321667,263.882778,no anomolies,1,1,9.9,8.72,18.62,1


In [2267]:
# Marked as retention
customer_retention[customer_retention.customer_unique_id == 'ca77025e7201e3b30c44b472ff346268']

,customer_unique_id,customer_order_id,purchase_date,order_status,approval_in-hours,delivery_in-hours,tag,number_item_types,num_items_in_order,total_item_price,total_shipping_price,total_order_value,retention_count
76388,ca77025e7201e3b30c44b472ff346268,6ccedfba5919d72fcc8c51bfa982de62,2018-06-01,delivered,15.886667,274.937500,no anomolies,1,1,74.99,19.21,94.20,7
76389,ca77025e7201e3b30c44b472ff346268,71f39c371308d132d7633895477dd307,2018-04-26,delivered,0.177500,342.462500,no anomolies,1,1,74.99,20.57,95.56,7
76390,ca77025e7201e3b30c44b472ff346268,852e5ea6e9d74416ddf88bdbdb3189b9,2018-05-03,delivered,0.347778,227.785556,no anomolies,1,4,43.96,148.16,192.12,7
76391,ca77025e7201e3b30c44b472ff346268,b145bff18e79ac4dfb3fb91e61906f38,2018-02-09,delivered,18.585556,433.108889,no anomolies,1,1,125.90,33.33,159.23,7
76392,ca77025e7201e3b30c44b472ff346268,c59e684f832f832056ceee2c310cfc7f,2018-01-25,delivered,0.123889,289.255278,no anomolies,1,1,59.90,18.96,78.86,7
76393,ca77025e7201e3b30c44b472ff346268,dc7dc47999d1b3c4c2f6a085a1a76eef,2018-02-17,delivered,0.756944,572.713611,no anomolies,3,6,313.94,113.42,427.36,7
76394,ca77025e7201e3b30c44b472ff346268,fc709ab645b71acd6046aeb03b590aa5,2017-10-09,delivered,14.863056,683.869167,no anomolies,1,1,269.90,19.17,289.07,7


EXPORT 

Before exporting the master data for Exploratory analysis, we need to drop some of the ID fields that were previouly used for merges. We will also explicitly define the order that the columns should show up

In [2268]:
# LIst the columns
master.columns

Index(['order_id', 'num_items_in_order', 'product_id_l', 'total_item_price',
       'total_shipping_price', 'total_order_value', 'product_id_r',
       'number_of_photos', 'category', 'customer_order_id', 'order_status',
       'tag', 'purchase_date', 'approval_in-hours', 'delivery_in-hours',
       'customer_unique_id', 'zipcode'],
      dtype='object')

In [2269]:
# Only keep the relevant columns and also specify order for readability
column_order = ['customer_unique_id', 'customer_order_id', 'order_status', 'purchase_date', 'num_items_in_order', \
                'total_item_price', 'total_shipping_price', 'total_order_value', 'approval_in-hours',\
                'delivery_in-hours', 'category', 'number_of_photos', 'zipcode', 'tag']

In [2270]:
# Now change column list and order using list above
master = master[column_order]
master_nonull = master_nonull[column_order]

In [2271]:
customer_retention.head()

,customer_unique_id,customer_order_id,purchase_date,order_status,approval_in-hours,delivery_in-hours,tag,number_item_types,num_items_in_order,total_item_price,total_shipping_price,total_order_value,retention_count
0,0000366f3b9a7992bf8c76cfdf3221e2,fadbb3709178fc513abc1b2670aa1ad2,2018-05-10,delivered,0.247500,153.869444,no anomolies,1,1,129.90,12.00,141.90,1
1,0000b849f77a49e4a4ce2b2a4ca5be3f,4cb282e167ae9234755102258dd52ee8,2018-05-07,delivered,7.238056,78.854167,no anomolies,1,1,18.90,8.29,27.19,1
2,0000f46a3911fa3c0805444483337064,9b3932a6253894a02c1df9d19004239f,2017-03-10,delivered,0.000000,617.562222,approval_issue approval-same-as-purchase,1,1,69.00,17.22,86.22,1
3,0000f6ccb0745a6a4b88665a16c9f078,914991f0c02ef0843c0e7010c819d642,2017-10-12,delivered,0.326667,480.890000,no anomolies,1,1,25.99,17.63,43.62,1
4,0004aac84e0df4da2b147fca70cf8255,47227568b10f5f58a524a75507e6992c,2017-11-14,delivered,0.352778,315.387222,no anomolies,1,1,180.00,16.89,196.89,1


And EXPORT into csv file

In [2272]:
master.to_csv('./bonsai_mock_data/output/bonsai_master.csv', index = False)
master_nonull.to_csv('./bonsai_mock_data/output/bonsai_product_analysis.csv', index = False)
customer_retention.to_csv('./bonsai_mock_data/output/bonsai_customer_analysis.csv', index = False)